In [257]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [258]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/1-training-data.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/20163727-test.csv', names=['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'y'])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      15 non-null     float64
 1   A2      15 non-null     float64
 2   A3      15 non-null     int64  
 3   A4      15 non-null     float64
 4   A5      15 non-null     int64  
 5   A6      15 non-null     float64
 6   A7      15 non-null     int64  
 7   A8      15 non-null     int64  
 8   y       15 non-null     int64  
dtypes: float64(4), int64(5)
memory usage: 1.2 KB


In [0]:
train.replace('?', np.nan, inplace=True)
from sklearn.impute import SimpleImputer
train['A1'] = pd.to_numeric(train['A1'], errors='coerce')
train['A2'] = pd.to_numeric(train['A2'], errors='coerce')
train['A3'] = pd.to_numeric(train['A3'], errors='coerce')
train['A4'] = pd.to_numeric(train['A4'], errors='coerce')
train['A5'] = pd.to_numeric(train['A5'], errors='coerce')
train['A6'] = pd.to_numeric(train['A6'], errors='coerce')
train['A7'] = pd.to_numeric(train['A7'], errors='coerce')
train['A8'] = pd.to_numeric(train['A8'], errors='coerce')

imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
train = pd.DataFrame(data=imp.fit_transform(train), columns=['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'y'] )

In [0]:
from sklearn.model_selection import train_test_split
X = train.drop('y', axis=1)
y = train['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

n_estimators = [800, 1200, 1500]
max_depth = [15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(RandomForestClassifier(), hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1,
                      scoring=f1)
bestF = gridF.fit(X_train, y_train)


Fitting 3 folds for each of 180 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  5.3min


In [0]:
bestF.best_params_

In [0]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_curve, average_precision_score
model = RandomForestClassifier(n_estimators=1200,
                               bootstrap=False,
                               criterion='entropy',
                               max_depth=15,
                                min_samples_split=2,
                                min_samples_leaf=1,
                                random_state=42,
                                n_jobs=-1, class_weight='balanced_subsample')
                               
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))
print(accuracy_score(y_test, predicted))

In [0]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.svm import SVC

mm_scaler = preprocessing.StandardScaler()
X_train_scaled = mm_scaler.fit_transform(X_train)
X_test_scaled = mm_scaler.fit_transform(X_test)

clf = SVC(kernel='linear')
clf.fit(X_train_scaled, y_train)
predicted = clf.predict(X_test_scaled)
print(classification_report(y_test, predicted))